## 고차함수
- 함수형 언어는 함수를 일급 시민(함수를 값처럼)으로 취급한다.
- 함수를 값처럼 사용할 수 있기 때문에 함수의 인자로 함수를 전달하거나 함수를 값으로 반환할 수 있다.
- 고차 함수는 함수를 매개 변수로 받거나 반환하는 함수를 얘기한다.
- 장점으로는 함수를 추상 / 구현으로 나눌 수 있어서 상황에 따라 교체가 유연해지고, 재사용성이 높다.

In [8]:
fun operate(
    x: Int,
    y: Int,
    // 매개변수 : Int,Int 반환값 : Int 에 해당하는 함수를 매개로 받을 수 있다.
    operation: (Int, Int) -> Int
): Int {
    // 매개로 받은 함수 실행
    return operation(x, y)
}

// operation에 대한 상세한 구현
fun add(a: Int, b: Int): Int = a + b
fun minus(a: Int, b: Int): Int = a - b
fun multiply(a: Int, b: Int): Int = a * b
fun divide(a: Int, b: Int): Int = a / b

val result = operate(1, 2, ::add)
val result2 = operate(1, 2, ::minus)
val result3 = operate(1, 2, ::multiply)
val result4 = operate(1, 2, ::divide)

// 람다를 넘기는 것도 가능
val result5 = operate(1, 2, { a, b -> a / b })
// 함수 마지막 매개가 람다일 때, 아래처럼 사용가능
var result6 = operate(1, 2) { a, b -> a / b }

println(result)
println(result2)
println(result3)
println(result4)
println(result5)

3
-1
2
0
0


## 함수 타입
- 함수의 모양을 표현하는 타입 ex) (Int, Int) -> Int
- 쉽게 생각하면 함수가 변수를 몇개 받는지, 변수는 타입이 뭔지, return은 뭘 반환하는지 표현 해주는 것.

In [1]:
// add 함수는 (Int, Int) -> Int 함수 타입을 가진다.
// Int 두 개 받아서 Int 만 념겨주면 되는 말이다.
val add: (Int, Int) -> Int = { a, b -> a + b }

// 매개와 반환이 없는 것도 표현 가능하다.
val void: () -> Unit = {}

val test: (Int) -> Int? = { a -> if (a > 0) a else null }

## 람다
- 함수형 타입의 구체적인 값을 만드는 방법은 이름을 지정하지 않는 **람다식**을 사용 하는 것이다.
- 아래 코드에서 보이는 { a, b -> a + b } 이 식이 **람다식**이다.
- 함수 정의와 달리 반환 타입 지정 X, 람다의 본문에서 반환타입 자동 추론된다. 반환 타입을 명시하지 못 한다.

In [13]:
fun aggregate(numbers: IntArray, operation: (Int, Int) -> Int): Int {
    var result = numbers.firstOrNull()?: 0

    for (i in numbers) {
        println(result)
        result = operation(result, i)
    }

    return result
}

fun su(numbers: IntArray) = aggregate(numbers, { a, b -> a + b })
fun mi(numbers: IntArray) = aggregate(numbers, { a, b -> a - b })

val result7 = su(intArrayOf(1, 2, 3))

println(result7)

1
2
4
7


## 익명 함수
- fun 키워드 다음에 파라메터가 오며 이름이 없다.
- 타입 추론 가능하다면 타입을 지정하지 않아도 된다.
- 익명 함수는 식이기 때문에 함수의 인자로 넘기거나 변수에 대입하는 일반 값처럼 쓸 수 있다.
- 식 본문, 블럭 본문 두가지 형태로 사용 가능하다.

In [15]:
fun sum(numbers: IntArray) = aggregate(numbers, fun(result, operation) = result + operation) // 식 본문
fun minus(numbers: IntArray) = aggregate(numbers, fun(result, operation): Int { return result - operation }) // 블록 본문

## 호출 가능 참조
- 함수나 프로퍼티를 객체처럼 참조할 수 있다. `::` 연산자 사용

In [19]:
fun double(x: Int): Int = x * 2
fun square(x: Int): Int = x * x

/**
* 함수 참조
* */
val action: (Int) -> Int = ::double // 함수 참조
val number = listOf(1, 2, 3)

val result = action(10)
val result2 = number.map(::square) // 함수 참조


/**
* 객체 참조
* */
class Person(val firstName: String) {
    fun fullName(message: String) = "$firstName $message"
}
val createPerson = ::Person // 객체 참조
val person = createPerson("gwangho", "yi")

val fullname = Person::fullName // 클래스 메서드 참조

println(fullname(person, "hello"))


/**
 * 생성자 참조
 * */
data class Person2(val name: String, val age: Int)
val personConstruct = ::Person2
val person2 = personConstruct("gwangho", 20)


/**
 * 프로퍼티 참조
 * */
class Person3(val firstName: String, val lastName: String) {
    val fullName: String
        get() = "$firstName $lastName"
}
val person3 = Person3("gwangho", "yi")
val firstNameRef = Person3::firstName
val lastNameRef = Person3::lastName
val fullNameRef = Person3::fullName

val firstName = firstNameRef.get(person3)
val lastName = lastNameRef.get(person3)
val fullName = fullNameRef.get(person3)



gwangho hello


## 인라인, 노인라인, 크로스 인라인 함수

- 인라인
    - 람다는 내부적으로 객체를 생성하고, 코드가 반복 실행되면 성능저하로 이어진다.
    - 인라인 함수를 사용하면 컴파일 타임에 코드를 직접 삽입하기 때문에 객체를 생성하지 않는다.
    - 즉, 람다 표현으로 작성 하면서도 성능 방어를 할 수 있다.
    - 쉽게 말해 함수의 매개 람다를 직접 실행 코드로 박아버리는 것이다.
    <br>
- 노인라인
    - 인라인 함수에서 특정 람다를 인라인하지 않는 것.
    - 람다를 노인라인으로 지정할 때는
        - 람다를 변수에 저장할 때
        - 다른 함수에 람다를 전달할 때
    - 인라인한 람다는 별도의 객체가 아니라 식이 되기 때문에
        - 인라인이 아닌 일반 함수에 전달 불가능
        - 변수에 저장 불가능

In [3]:
/**
* 인라인 함수
* */
inline fun repeat3Times(action: () -> Unit) {
    action()
    action()
    action()
}

repeat3Times {
    println("Hello")
}

// 위에 코드는 실제로 아래처럼 변환된다. action이 직접 삽입됨
// println("Hello")  // 첫 번째 action() 호출
// println("Hello")  // 두 번째 action() 호출
// println("Hello")  // 세 번째 action() 호출


fun repeat3Times2(action: Function0<Unit>) {
    action.invoke()
    action.invoke()
    action.invoke()
}

// inline 안 붙이고 생성하면 실제로는 아래처럼 객체가 생성된다. 반복문 3번 호출 시 객체 가 3개 생성된다.
val lambdaObject = object : Function0<Unit> {
    override fun invoke() {
        println("Hello")
    }
}

repeat3Times2(lambdaObject)


/**
 * 노인라인 함수
 * */
var lastAction: () -> Unit = {}

inline fun runAndMemorize(
    action: () -> Unit,
) {
    action()
    lastAction = action // 인라인 람다는 식이기 때문에 변수에 저장 X
}

fun noInline(action: () -> Unit) {
    action.invoke()
}

inline fun passLanda(action: () -> Unit) {
    noInline(action) // 인라인 함수가 아닌 일반 함수에는 인라인 함수를 넘길 수 없다
}

inline fun noPassLanda(
    noinline action1: () -> Unit,
    action2: () -> Unit
) {
    noInline { action1 }
    action2.invoke()
}


Hello
Hello
Hello
Hello
Hello
Hello
